all exercises based on [Introduction to Deep Learning for NLP](https://wikidocs.net/31698)

### BoW 만들기
1. 각 단어에 고유한 정수 인덱스 부여
2. 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터 생성

연습 문장<br>
**정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.**

In [2]:
# 입력 문서에 대한 단어 집합(vocabulary) 만들기 + 인덱스 할당 + BoW 생성
from konlpy.tag import Okt
import re
okt = Okt()

token = re.sub("(\.)","","정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다.") # 온점 제거
token = okt.morphs(token) # okt 형태소분석기로 토큰화한 걸 token에 담기

In [3]:
word2index = {}
bow = []
for word in token:
    if word not in word2index:
        word2index[word] = len(word2index)
        bow.insert(len(word2index)-1, 1) # dict는 없는 key를 인덱싱하고 값 넣으면 새로운 key&value가 생성되지만, list는 index out of range 오류 뜸. insert로 해줘야 함
    else:
        bow[word2index[word]] += 1
print(word2index)
print(bow)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
[1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


예전에 했던 정수 인코딩, one hot encoding 실습 시의 방식과는 달리, 빈도수는 인덱스 순서와는 아무런 상관 없음. 중요한 건 등장 빈도.

연습문장<br>
**소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.**

In [4]:
# 문장 1, 2를 합쳐서 BoW 만들기
token3= re.sub("(\.)", "", "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다. 소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")
token3 = okt.morphs(token3)


word2index3 = {}
bow3 = []
for word in token3:
    if word not in word2index3:
        word2index3[word] = len(word2index3)
        bow3.insert(len(word2index3)-1, 1)
    else:
        bow3[word2index3[word]] +=1

print(word2index3)
print(bow3)

{'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9, '는': 10, '주로': 11, '소비': 12, '상품': 13, '을': 14, '기준': 15, '으로': 16, '느낀다': 17}
[1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]


In [5]:
token2 = re.sub('(\.)', "", "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다.")
token2 = okt.morphs(token2)


# 문서 3에서 만든 단어 사전으로 문서 1, 2의 BoW을 만들어볼 것
bow2 = [0] * len(bow3)
bow = [0] * len(word2index3)

for word in token2:
    if word not in word2index3:
        word2index3[word] = len(word2index3)
        bow2.insert(len(word2index3)-1, 1)
    else:
        bow2[word2index3[word]] +=1
        
for word in token:
    if word not in word2index3:
        word2index3[word] = len(word2index3)
        bow.insert(len(word2index3)-1, 1)
    else:
        bow[word2index3[word]] +=1

In [6]:
print(bow)
print(bow2)

[1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2, 1, 1, 1]


### CountVectorizer 클래스로 BoW 생성

In [7]:
from sklearn.feature_extraction.text import CountVectorizer # 단어의 빈도를 count해 vector로 만들어줌.(영어) 손쉽게 만들 수 있음
corpus = ['you know I want your love. because I love you.']
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[1 1 2 1 2 1]]
{'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


In [9]:
# 불용어를 제거한 BoW

from sklearn.feature_extraction.text import CountVectorizer

text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words = ['the', 'a', 'an', 'is', 'not']) # 사용자 정의 불용어
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)


[[1 1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [11]:
# CountVectorizer가 제공하는 자체 불용어

vect = CountVectorizer(stop_words = "english") # everything 까지 불용어로 없앰
print(vect.fit_transform(text).toarray()) 
print(vect.vocabulary_)

[[1 1 1]]
{'family': 0, 'important': 1, 'thing': 2}


In [12]:
# nltk가 제공하는 자체 불용어
from nltk.corpus import stopwords

sw = stopwords.words('english')
vect = CountVectorizer(stop_words = sw)
print(vect.fit_transform(text).toarray())
print(vect.vocabulary_)


[[1 1 1 1]]
{'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
